# Terminal Commands, Cell Magics, and Shell Scripting

## `curl` and `wget`

`curl` is used in command lines or scripts to transfer data. curl is also used in cars, television sets, routers, printers, audio equipment, mobile phones, tablets, settop boxes, media players and is the Internet transfer engine for thousands of software applications in over ten billion installations [(Source)](https://curl.se/).

We use it here to take a peek at a `csv` from one of [Washington Post's GitHub repos](https://github.com/orgs/washingtonpost/repositories?q=&type=all&language=&sort=stargazers).

In [1]:
!curl https://raw.githubusercontent.com/washingtonpost/data-game-of-thrones-deaths/master/game-of-thrones-deaths-data.csv

`wget` is used for retrieving files using HTTP, HTTPS, FTP and FTPS, the most widely used Internet protocols. It is a non-interactive commandline tool, so it may easily be called from scripts, cron jobs, terminals without X-Windows support, etc. and provides the ability to recursively navigate entire directory structures to download files [(Source)](https://www.gnu.org/software/wget/).

We use it here to pull the `csv` file from the [Washington Post's GitHub repo](https://github.com/washingtonpost/data-game-of-thrones-deaths).

In [2]:
!wget https://raw.githubusercontent.com/washingtonpost/data-game-of-thrones-deaths/master/game-of-thrones-deaths-data.csv

--2022-09-29 16:04:28--  https://raw.githubusercontent.com/washingtonpost/data-game-of-thrones-deaths/master/game-of-thrones-deaths-data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8003::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 928822 (907K) [text/plain]
Saving to: ‘game-of-thrones-deaths-data.csv.1’

game-of-thrones-dea 100%[===================>] 907.05K  --.-KB/s    in 0.1s    

2022-09-29 16:04:29 (7.35 MB/s) - ‘game-of-thrones-deaths-data.csv.1’ saved [928822/928822]



## Using `cat`, `head`, and `tail` to inspect a `csv`

**Note:** The output from the first command gets funneled as the input into the second using the `|` operator, also known as the "pipe" operator.

In [3]:
%%bash
cat game-of-thrones-deaths-data.csv | head -n 2
cat game-of-thrones-deaths-data.csv | tail -n 1

order,season,episode,character_killed,killer,method,method_cat,reason,location,allegiance,importance
1,1,1,Waymar Royce,White Walker,Ice sword,Blade,Unknown,Beyond the Wall,"House Royce, Night’s Watch",2
6887,8,6,Daenerys Targaryen,Jon Snow,Knife,Blade,Stabbed in the stomach by Jon Snow after she “went mad” and burned down King’s Landing,King’s Landing,House Targaryen,4

## Using the `grep` command to save the output subset of a `csv`

We use `grep` below to parse only the rows containing the string "Arya Stark" and output them as a separate `csv`:

In [4]:
%%bash

grep -a "Arya Stark" game-of-thrones-deaths-data.csv > arya-kills.csv

The output `csv` doesn't contain any column headers, so we make sure to pull them in from the original `csv` as we read in the file:

In [5]:
import pandas as pd

pd.read_csv('arya-kills.csv',
            names = pd.read_csv('game-of-thrones-deaths-data.csv').columns.tolist(),
           ).head(1)

,order,season,episode,character_killed,killer,method,method_cat,reason,location,allegiance,importance
0,46,1,8,Stableboy,Arya Stark,Sword (Needle),Blade,Tried to grab Arya,King’s Landing,Smallfolk,1


## Using the `%%bash` cell magic to execute a Python script

In [6]:
%%bash
python quick_count.py

season
1      59
2     130
3      87
4     181
5     246
6     540
7    1096
8    4548
Name: character_killed, dtype: int64


## Using the `%%writefile` cell magic to output a file

In this case, we're outputing a shell script called `in case of emergency.sh` (note the `.sh` file extension denotes a shell scipt) 

In [7]:
%%writefile "in_case_of_emergency.sh"
#!/bin/bash
set -e
set -u
set -o pipefail

rm -rf DS-Deloitte-07062022
git clone git@github.com:flatiron-school/DS-Deloitte-07062022.git
cd DS-Deloitte-07062022

Overwriting in_case_of_emergency.sh


## Use this in all your shell scripts!

```
#!/bin/bash
set -e
set -u
set -o pipefail
```

The first line (`#!/bin/bash`) is called the `shebang`, and it indicates the path to the interpreter used to execute this script. This is necessary any time execution of the script is required. Regardless of how you plan to run your Bash script, it’s best to include a shebang line. By default, a shell script containing a command that fails (exits with a nonzero exit status) will not cause the entire shell script to exit — the shell script will just continue on to the next line. This is not a desirable behavior; we always want errors to be loud and noticeable!

`set -e` prevents this, by terminating the script if any command exited with a nonzero exit status. Note, however, that `set -e` has complex rules to accommodate cases when a nonzero exit status indicates “false” rather than failure. For example, `test -d file.txt` will return a nonzero exit status if its argument is not a directory, but in this context this isn’t meant to represent an error. `set -e` ignores nonzero statuses in if conditionals for this reason (we’ll discuss this later). Also, `set -e` ignores all exit statuses in Unix pipes except the last one — this relates to `set -o pipefail`, which we'll circle back to.

`set -u` fixes another unfortunate default behavior of Bash scripts: any command containing a reference to an unset variable name will still run. As a horrifying example of what this can lead to, consider: `rm -rf $TEMP_DIR/*`. If the shell variable `$TEMP_DIR` isn’t set, Bash will still substitute its value (which is nothing) in place of it. The end result is `rm -rf /*`! You can see this for yourself: `$ echo "rm $NOTSET/blah" rm /blah set -u` prevents this type of error by aborting the script if a variable’s value is unset. As just discussed, `set -e` will cause a script to abort if a nonzero exit status is encountered, with some exceptions. One such exception is if a program run in a Unix pipe exited unsuccessfully; unless this program was the last program in the pipe, this would not cause the script to abort even with `set -e`.

Including `set -o pipefail` will prevent this undesirable behavior — any program that returns a nonzero exit status in the pipe will cause the entire pipe to return a nonzero status. With `set -e` enabled, too, this will lead the script to abort.

These three options are the first layer of protection against Bash scripts with silent errors and unsafe behavior. Unfortunately, Bash is a fragile language, and we need to mind a few other oddities to use it safely.

In [8]:
%%writefile quick_viz.py
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('game-of-thrones-deaths-data.csv')
print(df.groupby('season')['character_killed'].count().plot(kind='bar'))

plt.show()

Overwriting quick_viz.py


Now go try it out using the `in case of emergency.sh` script -- move it in the same directory as your cloned version of the `DS-Deloitte-07062022` repo (i.e., your Desktop) and run it using any if the following ways:

* `./in_case_of_emergency.sh`
* `sh in_case_of_emergency.sh`
* `bash in_case_of_emergency.sh`

**Note:** *You'll first need to quickly give the file the correct permission to execute using this command:* `chmod +x in_case_of_emergency.sh`

You'll notice that it performs the steps of recursively deleting the contents of your cloned version of the course repo and pulls down a fresh clone. This is particularly useful for when you are having a hard time pulling down fresh content from our repo! 😎

However, you'll notice that it didn't automatically `cd` into the repo, which was the final step we told it to perform. This is because the script ran in a sub-shell and couldn't `cd` into the location you specified. As we create shell scripts that go to varying file locations to run commands, this is a way of thinking and awareness that we must develop, so please tinker and become more familiar with the command line -- it's your friend and can really boost your productivity!

# Level-Up Content!

Run the following cell, open your terminal, `cd` into the folder containing this notebook, and run the `quick_viz.py` script using the command `python quick_viz.py`:

In [9]:
%%writefile quick_viz.sh
#!/bin/bash
set -e
set -u
set -o pipefail
python quick_viz.py

Overwriting quick_viz.sh


Now, notice that when we go to run `quick_viz.sh`, we get the following error: `zsh: permission denied: ./quick_viz.sh`. Remember, we must give it permission to run using the command `chmod +x quick_viz.sh`.

Congrats! You just wrapped a Python script in a shell script and executed it!! Hopefully you're beginning to see how scripting languages are like a glue that can hold various processes together in the form of a coherent, reproducible, automated workflow.

## Using `sed` and `awk` to clean data

What if we wanted to clean the following `csv` using the terminal?

In [10]:
%%bash
cat ages.csv

age,value
"33","5943"
"32","543"
"34","93"
"39","5943"
"36","9943"
"38","8943"


We could use `cat` in conjunction with `sed` to augment the data structure (and save the resulting output, if we wanted to):

In [11]:
%%bash

cat ages.csv | sed 's/"//g'

age,value
33,5943
32,543
34,93
39,5943
36,9943
38,8943


What about the mixed units in the data structure below?

In [12]:
%%bash
cat temps.txt

temp,unit
26.1,C
78.1,F
23.1,C
25.7,C
76.3,F
77.3,F
24.2,C
79.3,F
27.9,C
75.1,F
25.9,C
79.0,F


Using `cat` and `awk` we can augment the data structure thusly:

In [13]:
%%bash
cat temps.txt | awk -F',' '{if($2=="F")print (($1-32)*5/9)",C";else print $1","$2}'

temp,unit
26.1,C
25.6111,C
23.1,C
25.7,C
24.6111,C
25.1667,C
24.2,C
26.2778,C
27.9,C
23.9444,C
25.9,C
26.1111,C


## Installing Jupyter Extensions

Installing Jupyter extensions can also increase your productivity -- here are some [ideas](https://www.analyticsvidhya.com/blog/2022/02/11-extensions-to-power-up-your-jupyter-notebook/) and [installation help](https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/install.html) to get you started! Running the following cells and restarting Jupyter should also expedite the process for you:

In [14]:
!pip install jupyter_contrib_nbextensions

In [15]:
%%bash

jupyter contrib nbextension install --user

[I 16:04:32 InstallContribNbextensionsApp] jupyter contrib nbextension install --user
[I 16:04:32 InstallContribNbextensionsApp] Installing jupyter_contrib_nbextensions nbextension files to jupyter data directory
[I 16:04:32 InstallContribNbextensionsApp] Installing /Users/nickmccarty/opt/anaconda3/lib/python3.9/site-packages/jupyter_contrib_nbextensions/nbextensions/rubberband -> rubberband
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/rubberband/icon.png
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/rubberband/main.css
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/rubberband/readme.md
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/rubberband/main.js
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/rubberband

[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/table_beautifier/main.css
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/table_beautifier/README.md
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/table_beautifier/main.js
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/table_beautifier/table_beautifier.yaml
[I 16:04:32 InstallContribNbextensionsApp] - Validating: OK
[I 16:04:32 InstallContribNbextensionsApp] Installing /Users/nickmccarty/opt/anaconda3/lib/python3.9/site-packages/jupyter_contrib_nbextensions/nbextensions/freeze -> freeze
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/freeze/icon.png
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/freeze/config.y

[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/scratchpad/scratchpad.yaml
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/scratchpad/README.md
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/scratchpad/main.js
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/scratchpad/scratchpad.css
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/scratchpad/demo.gif
[I 16:04:32 InstallContribNbextensionsApp] - Validating: OK
[I 16:04:32 InstallContribNbextensionsApp] Installing /Users/nickmccarty/opt/anaconda3/lib/python3.9/site-packages/jupyter_contrib_nbextensions/nbextensions/toc2 -> toc2
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/toc2/toc2.yaml
[I 16:04:32 InstallContr

[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/snippets_menu/snippets_submenus_python/matplotlib.js
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/snippets_menu/snippets_submenus_python/sympy_functions.js
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/snippets_menu/snippets_submenus_python/h5py.js
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/snippets_menu/snippets_submenus_python/scipy_special.js
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/snippets_menu/snippets_submenus_python/python_regex.js
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/snippets_menu/snippets_submenus_python/python.js
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/ni

[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/keyboard_shortcut_editor/readme_menu_item.png
[I 16:04:32 InstallContribNbextensionsApp] - Validating: OK
[I 16:04:32 InstallContribNbextensionsApp] Installing /Users/nickmccarty/opt/anaconda3/lib/python3.9/site-packages/jupyter_contrib_nbextensions/nbextensions/help_panel -> help_panel
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/help_panel/icon.png
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/help_panel/help_panel_ext.png
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/help_panel/readme.md
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/help_panel/help_panel_ext_fullscreen.png
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jup

[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/codefolding/magic-unfolded.png
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/codefolding/foldgutter.css
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/codefolding/magic-folded.png
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/codefolding/codefolding.yaml
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/codefolding/codefolding_editor.png
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/codefolding/codefolding_indent_unfolded.png
[I 16:04:32 InstallContribNbextensionsApp] - Validating: OK
[I 16:04:32 InstallContribNbextensionsApp] Installing /Users/nickmccarty/opt/anaconda3/lib/python3.9/site-packages/jupyter_cont

[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/contrib_nbextensions_help_item/README.md
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/contrib_nbextensions_help_item/main.js
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/contrib_nbextensions_help_item/contrib_nbextensions_help_item.yaml
[I 16:04:32 InstallContribNbextensionsApp] - Validating: OK
[I 16:04:32 InstallContribNbextensionsApp] Installing /Users/nickmccarty/opt/anaconda3/lib/python3.9/site-packages/jupyter_contrib_nbextensions/nbextensions/datestamper -> datestamper
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/datestamper/icon.png
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/datestamper/main.yaml
[I 16:04:32 InstallContribNbextensionsApp] Up to da

[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/varInspector/icon.png
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/varInspector/jquery.tablesorter.min.js
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/varInspector/varInspector.yaml
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/varInspector/main.css
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/varInspector/var_list.py
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/varInspector/README.md
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/varInspector/main.js
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbexte

[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/code_prettify/code_prettify.js
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/code_prettify/autopep8.js
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/code_prettify/demo-R.gif
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/code_prettify/isort.js
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/code_prettify/code_prettify.yaml
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/code_prettify/2to3.js
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/code_prettify/autopep8.yaml
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/n

[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/highlight_selected_word/main.js
[I 16:04:32 InstallContribNbextensionsApp] - Validating: OK
[I 16:04:32 InstallContribNbextensionsApp] Installing /Users/nickmccarty/opt/anaconda3/lib/python3.9/site-packages/latex_envs/static -> latex_envs
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/latex_envs/icon.png
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/latex_envs/latex_envs.yaml
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/latex_envs/configuration_toolbar.png
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/latex_envs/bibInNb4.js
[I 16:04:32 InstallContribNbextensionsApp] Up to date: /Users/nickmccarty/Library/Jupyter/nbextensions/latex_envs/example_user_envs.json
[I

## TidyTuesday

In the spirit of what the R community has codified in their [TidyTuesday](https://www.tidytuesday.com/) ritual, one of my Flatiron colleagues put together a [repo](https://github.com/learn-co-curriculum/tidytuesday) into which each new TidyTuesday challenge is pushed, thereby providing myriad opportunities to practice, pratice, practice!